In [163]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

In [164]:
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}

In [165]:
W=set(list(range(0,12))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
 #set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)

In [166]:
# Create an empty model
m1= gp.Model(name="NFL1")

In [167]:
#home=len(E.columns)
#away=len(E.columns)
weeks=13
#if team h plays at home against team j at week k
x = m1.addVars(T, T, W, vtype = GRB.BINARY, name = "x")

In [168]:
# Objective is to minimize the total distance of all teams
#obj = sum(sum(x[w,h,a]*2*E.iloc[h,a] for h in range(home) for a in range(away)) for w in range(weeks))
obj = sum(sum(x[h,a,w]*2*E.iloc[h,a] for w in W) for h in T for a in T)

In [169]:
#obj = sum(E.iloc[h,a]*x[h,a,w] for h in T for a in T for w in W)*2

In [170]:
#The season was limited to 12 weeks.
con1 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for w in W for h in T) == 12 
                     for a in T)

In [171]:
#each team would play once per week
con2 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for h in T) == 1 
                    for a in T for w in W)

In [172]:
#add constraints that all 12 games that a team played would need to be against a different opponent 
con3 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for w in W) <= 1
                    for h in T for a in T)

In [173]:
#add constraints that each team would play at most 6 home games
con4 = m1.addConstrs(sum(x[h,a,w] for h in T for w in W) <= 6 for a in T)

In [174]:
#add constraints that no team can plays itself
con0 = m1.addConstrs((x[h,h,w] == 0 for h in T for w in W))

In [175]:
#add constraints game pattern 
con_h = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] + x[h,a,w+2] for a in T) <= 2 for h in T for w in range(8,10))

In [176]:
con_h1 = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] + x[h,a,w+2] for a in T) <= 2 for h in T for w in range(6))

In [177]:
con_a = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] + x[h,a,w+2] for a in T) >= 1 for h in T for w in range(8,10))

In [178]:
con_a = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] + x[h,a,w+2] for a in T) >= 1 for h in T for w in range(6))

In [ ]:
#home games
for i in range(T):
    for k in range(6):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
    for k in range(8,10):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
        m1.addConstr(home<=2)

        
for i in range(N):
    for k in range(6):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
    for k in range(8,10):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
        m1.addConstr(home>=1)

In [179]:
#add constraints bye week
con_bye = m1.addConstrs(x[h,a,8] == 0 for h in T for a in T)

In [155]:
#Update the week of plays 
con_8week = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for a in T) == 1 for h in T 
                     for w in range(0,8))

con_13week = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for a in T) ==1 for h in T 
                     for w in range(9,13))


KeyError: (0, 0, 12)

In [180]:
#add constraints that each team would play at most 6 home games
con_BI = m1.addConstrs(x[h,a,w] <=1 for h in T for a in T for w in W)

In [181]:
#add constraints that each team would play at most 6 home games
con_BI = m1.addConstrs(x[h,a,w] >=0 for h in T for a in T for w in W)

In [182]:
m1.setObjective(obj, GRB.MINIMIZE)
m1.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15984 rows, 6912 columns and 89856 nonzeros
Model fingerprint: 0x90784642
Variable types: 0 continuous, 6912 integer (6912 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 14664 rows and 840 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.12 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
